In [1]:
## change to our functinon
from stage import getStage, fullPrint
import numpy as np
import pandas as pd
from datetime import date, timedelta
import yahoo_fin.stock_info as yf
import random
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
def evalOneMax(individual):
    print(individual[0])
    winningTrade = 0
    totalTrade = 0
    transaction = yf.get_data('AAPL', start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    tickerIndex = random.randint(0,len(yf.tickers_nasdaq())-1)
    for symbol in yf.tickers_nasdaq():
        df = getStage(symbol,individual[0])
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        drift = -1
        for index, element in df.iterrows():
            if drift == -1:
                drift = index.weekday()
            index = index - timedelta(days= drift)
            index = index.strftime('%Y-%m-%d')
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                if element.Stage == "Buy" and not inStage:
                    buyPrice = element.close
                transaction.loc[index]['holding'].append((symbol,element.close,0))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                if element.Stage == "Buy" and inStage:
                    secondBuyPrice = element.close
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                    buyTwice = True
                inStage = True
                continue
            if element.Stage == "Sell":
                transaction.loc[index]['holding'].append((symbol,element.close,-1))
                returnRate = element.close / buyPrice
                if returnRate > 1:
                    winningTrade += 1
                    totalTrade += 1
                else:
                    totalTrade += 1
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,-1))
                    returnRate = element.close / secondBuyPrice
                    if returnRate > 1:
                        winningTrade += 1
                        totalTrade += 1
                    else:
                        totalTrade += 1
                buyPrice = 0
                secondBuyPrice = 0
                inStage = False
                buyTwice = False
    total = 100
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    stockHolding  = 0
    for i in transactionCopy.iterrows():
        stockHolding += len(i[1]['holding'])
    print(stockHolding/(transactionCopy.shape[0]))
    print(transactionCopy.iloc[-1]['total'])
    if totalTrade==0:
        print(winningTrade, totalTrade, 0)
    else:
        print(winningTrade, totalTrade, winningTrade/totalTrade)
    transactionCopy.to_csv("transaction/checkBug.csv")
    transactionCopy.to_pickle("transaction/checkBug.pkl")
    return transactionCopy.iloc[-1]['total']
evalOneMax([[0.3318662099030736, 1.7347893537959698, 0.1916285863344682, 0.9350007642794158, 1.3283762780182027, 1.0069983167660093, 0.9708546875087183, 0.8212695161601586, 1.4036713065821127]])

[0.3318662099030736, 1.7347893537959698, 0.1916285863344682, 0.9350007642794158, 1.3283762780182027, 1.0069983167660093, 0.9708546875087183, 0.8212695161601586, 1.4036713065821127]
19.740609496810773
13591.246380139364
187 530 0.35283018867924526


13591.246380139364

In [2]:
from stage import getFullDf
getFullDf("ABUS",[0.3318662099030736, 1.7347893537959698, 0.1916285863344682, 0.9350007642794158, 1.3283762780182027, 1.0069983167660093, 0.9708546875087183, 0.8212695161601586, 1.4036713065821127]).to_csv("ABUS.csv")

6484


In [3]:
# from stage import getStage
# getStage("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544])
from stage import getFullDf
getFullDf("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544,1.5]).to_csv("result.csv")

6484
